# OmniGenome - A Demonstration based on RNA Secondary Structure Prediction
GitHub: https://github.com/yangheng95/OmniGenome
OmniGenome Hub: Huggingface Spaces

## Introduction
OmniGenome is a comprehensive package designed for pretrained genomic foundation models (FMs) development and FM benchmark. 
OmniGenome have the following key features:
- Automated genomic FM benchmarking on public genomic datasets
- Scalable genomic FM training and fine-tuning on genomic tasks
- Diversified genomic FMs implementation
- Easy-to-use pipeline for genomic FM development with no coding expertise required
- Accessible OmniGenome Hub for sharing FMs, datasets, and pipelines
- Extensive documentation and tutorials for genomic FM development

We begin to introduce OmniGenome by delivering a demonstration to train a model to predict RNA secondary structures. The dataset used in this demonstration is the bpRNA dataset which contains RNA sequences and their corresponding secondary structures. The secondary structure of an RNA sequence is a set of base pairs that describe the folding of the RNA molecule. The secondary structure of an RNA sequence is important for understanding the function of the RNA molecule. In this demonstration, we will train a model to predict the secondary structure of an RNA sequence given its primary sequence.

## Requirements
OmniGenome requires the following recommended dependencies:
- Python 3.9+
- PyTorch 2.0.0+
- Transformers 4.37.0+
- Pandas 1.3.3+
- Others in case of specific tasks

## Fine-tuning Genomic FMs for RNA Secondary Structure Prediction

### Step 1: Import Libraries

In [2]:
import autocuda
import torch
from metric_visualizer import MetricVisualizer

from omnigenome import OmniGenomeDatasetForTokenClassification
from omnigenome import ClassificationMetric
from omnigenome import OmniSingleNucleotideTokenizer, OmniKmersTokenizer
from omnigenome import OmniGenomeModelForTokenClassification
from omnigenome import Trainer

### Step 2: Define and Initialize the Tokenizer

In [3]:
# Predefined dataset label mapping
label2id = {"(": 0, ")": 1, ".": 2}

# The is FM is exclusively powered by the OmniGenome package
model_name_or_path = "anonymous8/OmniGenome-52M"

# Generally, we use the tokenizers from transformers library, such as AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# However, OmniGenome provides specialized tokenizers for genomic data, such as single nucleotide tokenizer and k-mers tokenizer
# we can force the tokenizer to be used in the model
tokenizer = OmniSingleNucleotideTokenizer.from_pretrained(model_name_or_path)

### Step 3: Define and Initialize the Model

In [4]:
# We have implemented a diverse set of genomic models in OmniGenome, please refer to the documentation for more details
ssp_model = OmniGenomeModelForTokenClassification(
    model_name_or_path,
    tokenizer=tokenizer,
    label2id=label2id,
)

### Step 4: Define and Load the Dataset

In [5]:
# necessary hyperparameters
epochs = 1
learning_rate = 2e-5
weight_decay = 1e-5
batch_size = 8
max_length = 512
seeds = [45]  # Each seed will be used for one run


# Load the dataset according to the path
train_file = "toy_datasets/train.json"
test_file = "toy_datasets/test.json"
valid_file = "toy_datasets/valid.json"

train_set = OmniGenomeDatasetForTokenClassification(
    data_source=train_file,
    tokenizer=tokenizer,
    label2id=label2id,
    max_length=max_length,
)
test_set = OmniGenomeDatasetForTokenClassification(
    data_source=test_file,
    tokenizer=tokenizer,
    label2id=label2id,
    max_length=max_length,
)
valid_set = OmniGenomeDatasetForTokenClassification(
    data_source=valid_file,
    tokenizer=tokenizer,
    label2id=label2id,
    max_length=max_length,
)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

### Step 5: Define the Metrics
We have implemented a diverse set of genomic metrics in OmniGenome, please refer to the documentation for more details.
Users can also define their own metrics by inheriting the `OmniGenomeMetric` class. 
The `compute_metrics` can be a metric function list and each metric function should return a dictionary of metrics.

In [6]:
compute_metrics = [
    ClassificationMetric(ignore_y=-100).accuracy_score,
    ClassificationMetric(ignore_y=-100, average="macro").f1_score,
]

## Step 6: Define and Initialize the Trainer

In [7]:
# Initialize the MetricVisualizer for logging the metrics
mv = MetricVisualizer(name="OmniGenome-52M-SSP")

for seed in seeds:
    optimizer = torch.optim.AdamW(
        ssp_model.parameters(), lr=learning_rate, weight_decay=weight_decay
    )
    trainer = Trainer(
        model=ssp_model,
        train_loader=train_loader,
        eval_loader=valid_loader,
        test_loader=test_loader,
        batch_size=batch_size,
        epochs=epochs,
        optimizer=optimizer,
        compute_metrics=compute_metrics,
        seeds=seed,
        device=autocuda.auto_cuda(),
    )

    metrics = trainer.train()
    test_metrics = metrics["test"][-1]
    mv.log(model_name_or_path.split("/")[-1], "F1", test_metrics["f1_score"])
    mv.log(
        model_name_or_path.split("/")[-1],
        "Accuracy",
        test_metrics["accuracy_score"],
    )
    print(metrics)
    mv.summary()

### Step 7. Experimental Results Visualization
The experimental results are visualized in the following plots. The plots show the F1 score and accuracy of the model on the test set for each run. The average F1 score and accuracy are also shown.

|### Step 8. Model Checkpoint for Sharing
The model checkpoint can be saved and shared with others for further use. The model checkpoint can be loaded using the following code:

**Regular checkpointing and resuming are good practices to save the model at different stages of training.**

In [8]:
path_to_save = "OmniGenome-52M-SSP"
ssp_model.save(path_to_save, overwrite=True)

# Load the model checkpoint
ssp_model = ssp_model.load(path_to_save)
results = ssp_model.inference("CAGUGCCGAGGCCACGCGGAGAACGAUCGAGGGUACAGCACUA")
print(results["predictions"])
print("logits:", results["logits"])


# What if someone doesn't know how to initialize the model?

In [9]:
# We can load the model checkpoint using the ModelHub
from omnigenome import ModelHub

ssp_model = ModelHub.load("OmniGenome-52M-SSP")
results = ssp_model.inference("CAGUGCCGAGGCCACGCGGAGAACGAUCGAGGGUACAGCACUA")
print(results["predictions"])
print("logits:", results["logits"])

## Step 8. Model Inference

In [10]:
examples = [
    "GCUGGGAUGUUGGCUUAGAAGCAGCCAUCAUUUAAAGAGUGCGUAACAGCUCACCAGC",
    "AUCUGUACUAGUUAGCUAACUAGAUCUGUAUCUGGCGGUUCCGUGGAAGAACUGACGUGUUCAUAUUCCCGACCGCAGCCCUGGGAGACGUCUCAGAGGC",
]

results = ssp_model.inference(examples)
structures = ["".join(prediction) for prediction in results["predictions"]]
print(results)
print(structures)

### Step 9. Pipeline Creation
The OmniGenome package provides pipelines for genomic FM development. The pipeline can be used to train, fine-tune, and evaluate genomic FMs. The pipeline can be used with a single command to train a genomic FM on a dataset. The pipeline can also be used to fine-tune a pre-trained genomic FM on a new dataset. The pipeline can be used to evaluate the performance of a genomic FM on a dataset. The pipeline can be used to generate predictions using a genomic FM.

In [11]:
from omnigenome import Pipeline, PipelineHub

pipeline = Pipeline(
    name="OmniGenome-52M-SSP-Pipeline",
    # model_name_or_path="OmniGenome-52M-SSP",  # The model name or path can be specified
    # tokenizer="OmniGenome-52M-SSP",  # The tokenizer can be specified
    model_name_or_path=ssp_model,
    tokenizer=ssp_model.tokenizer,
    datasets={
        "train": "toy_datasets/train.json",
        "test": "toy_datasets/test.json",
        "valid": "toy_datasets/valid.json",
    },
    trainer=trainer,
    device=ssp_model.model.device,
)

### Using the Pipeline

In [ ]:
results = pipeline(examples[0])
print(results)

pipeline.train()

pipeline.save("OmniGenome-52M-SSP-Pipeline", overwrite=True)

pipeline = PipelineHub.load("OmniGenome-52M-SSP-Pipeline")
results = pipeline(examples)
print(results)

## Web Demo for RNA Secondary Structure Prediction

In [2]:
import numpy as np
import json
import gradio as gr
import RNA


def ss_validity_loss(rna_strct):
    dotCount = 0
    leftCount = 0
    rightCount = 0
    unmatched_positions = []  # 用于记录未匹配括号的位置
    uncoherentCount = 0
    prev_char = ""
    for i, char in enumerate(rna_strct):
        if prev_char != char:
            uncoherentCount += 1
        prev_char = char

        if char == "(":
            leftCount += 1
            unmatched_positions.append(i)  # 记录左括号位置
        elif char == ")":
            if leftCount > 0:
                leftCount -= 1
                unmatched_positions.pop()  # 移除最近的左括号位置
            else:
                rightCount += 1
                unmatched_positions.append(i)  # 记录右括号位置
        elif char == ".":
            dotCount += 1
        else:
            raise ValueError(f"Invalid character {char} in RNA structure")
    match_loss = (leftCount + rightCount) / (len(rna_strct) - dotCount + 1e-5)
    return match_loss


def find_invalid_ss_positions(rna_strct):
    left_brackets = []  # 存储左括号的位置
    right_brackets = []  # 存储未匹配的右括号的位置
    for i, char in enumerate(rna_strct):
        if char == "(":
            left_brackets.append(i)
        elif char == ")":
            if left_brackets:
                left_brackets.pop()  # 找到匹配的左括号，从列表中移除
            else:
                right_brackets.append(i)  # 没有匹配的左括号，记录右括号的位置
    return left_brackets + right_brackets


def fold(rna_sequence):
    ref_struct = RNA.fold(rna_sequence)[0]
    RNA.svg_rna_plot(rna_sequence, ref_struct, f"real_structure.svg")

    pred_structure = "".join(ssp_model.inference(rna_sequence)["predictions"])
    print(pred_structure)
    if ss_validity_loss(pred_structure) == 0:
        RNA.svg_rna_plot(rna_sequence, pred_structure, f"predicted_structure.svg")
        return (
            ref_struct,
            pred_structure,
            "real_structure.svg",
            "predicted_structure.svg",
        )
    else:
        # return blank image of predicted structure
        # generate a blank svg image
        with open("predicted_structure.svg", "w") as f:
            f.write(
                '<svg xmlns="http://www.w3.org/2000/svg" width="100" height="100"></svg>'
            )
        return (
            ref_struct,
            pred_structure,
            "real_structure.svg",
            "predicted_structure.svg",
        )


def repair_rna_structure(rna_sequence, invalid_struct):
    try:
        invalid_ss_positions = find_invalid_ss_positions(invalid_struct)
        for pos_idx in invalid_ss_positions:
            if invalid_struct[pos_idx] == "(":
                invalid_struct = (
                    invalid_struct[:pos_idx] + "." + invalid_struct[pos_idx + 1 :]
                )
            else:
                invalid_struct = (
                    invalid_struct[:pos_idx] + "." + invalid_struct[pos_idx + 1 :]
                )

        best_pred_struct = invalid_struct
        RNA.svg_rna_plot(rna_sequence, best_pred_struct, f"best_pred_struct.svg")
        return best_pred_struct, "best_pred_struct.svg"
    except Exception as e:
        with open("best_pred_struct.svg", "w") as f:
            f.write(
                '<svg xmlns="http://www.w3.org/2000/svg" width="100" height="100"></svg>'
            )
        return e, "best_pred_struct.svg"


def sample_rna_sequence():
    example = examples[np.random.randint(0, len(examples))]
    RNA.svg_rna_plot(example["seq"], example["label"], f"annotated_structure.svg")

    return example["seq"], example["label"], "annotated_structure.svg"


# 定义界面
with gr.Blocks() as demo:
    gr.Markdown("### RNA Secondary Structure Prediction")

    with gr.Row():
        with gr.Row():
            rna_input = gr.Textbox(
                label="RNA Sequence", placeholder="Enter RNA sequence here..."
            )
        with gr.Row():
            strcut_input = gr.Textbox(
                label="Annotated Secondary Structure",
                placeholder="Enter RNA secondary structure here...",
            )

    with gr.Row():
        #     examples = [
        #     ["GCGUCACACCGGUGAAGUCGCGCGUCACACCGGUGAAGUCGC"],
        #     ["GCUGGGAUGUUGGCUUAGAAGCAGCCAUCAUUUAAAGAGUGCGUAACAGCUCACCAGCGCUGGGAUGUUGGCUUAGAAGCAGCCAUCAUUUAAAGAGUGCGUAACAGCUCACCAGC"],
        #     ["GGCUGGUCCGAGUGCAGUGGUGUUUACAACUAAUUGAUCACAACCAGUUACAGAUUUCUUUGUUCCUUCUCCACUCCCACUGCUUCACUUGACUAGCCUU"],
        # ]
        #     gr.Examples(examples=examples, label="Examples", inputs=[rna_input])
        with open("toy_datasets/test.json", "r") as f:
            examples = []
            for line in f:
                examples.append(json.loads(line))

        sample_button = gr.Button("Sample a RNA Sequence from RNAStrand2 testset")

    with gr.Row():
        submit_button = gr.Button("Run Prediction")

    with gr.Row():
        ref_structure_output = gr.Textbox(
            label="Secondary Structure by ViennaRNA", interactive=False
        )

    with gr.Row():
        pred_structure_output = gr.Textbox(
            label="Secondary Structure by Model", interactive=False
        )

    with gr.Row():
        anno_structure_output = gr.Image(
            label="Annotated Secondary Structure", show_share_button=True
        )
        real_image = gr.Image(
            label="Secondary Structure by ViennaRNA", show_share_button=True
        )
        predicted_image = gr.Image(
            label="Secondary Structure by Model", show_share_button=True
        )

    with gr.Row():
        repair_button = gr.Button("Run Prediction Repair")

    submit_button.click(
        fn=fold,
        inputs=rna_input,
        outputs=[
            ref_structure_output,
            pred_structure_output,
            real_image,
            predicted_image,
        ],
    )

    repair_button.click(
        fn=repair_rna_structure,
        inputs=[rna_input, pred_structure_output],
        outputs=[pred_structure_output, predicted_image],
    )

    sample_button.click(
        fn=sample_rna_sequence, outputs=[rna_input, strcut_input, anno_structure_output]
    )
demo.launch()

### Conclusion
In this demonstration, we have shown how to fine-tune a genomic foundation model for RNA secondary structure prediction using the OmniGenome package. We have also shown how to use the trained model for inference and how to create a web demo for RNA secondary structure prediction. We hope this demonstration will help you get started with genomic foundation model development using OmniGenome.